## Import dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv("./dataset/UNSW_NB15_training-set.csv")
test = pd.read_csv("./dataset/UNSW_NB15_testing-set.csv")

## Preprocessing

In [2]:
data = pd.concat([train, test]).reset_index(drop=True)
data = data.drop(columns=["id"], axis=1)

# get numeric column names
num_cols = data._get_numeric_data().columns
# subract numeric columns from all columns to get categorical columns 
cat_cols = list(set(data.columns) - set(num_cols))

data[cat_cols] = data[cat_cols].astype('category')

data[cat_cols] = data[cat_cols].apply(lambda x: x.cat.codes)
data.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,119,0,5,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,6,0
1,0.000008,119,0,5,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,6,0
2,0.000005,119,0,5,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,6,0
3,0.000006,119,0,5,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,6,0
4,0.000010,119,0,5,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,6,0


In [3]:
from sklearn.model_selection import train_test_split

X = data.drop(["attack_cat", "label"], axis=1)
Y = data.loc[:, ["label"]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


In [4]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier()

mlp_clf.fit(X_train, np.ravel(Y_train))

mlp_clf.score(X_test, np.ravel(Y_test))

0.761868921477544

In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'activation': ['relu', 'logistic', 'tanh'],
    'hidden_layer_sizes': [(100,), (50, 50, 50)],
    'solver': ['sgd', 'adam'],
    # 'learning_rate': ['adaptive', 'invscaling', 'constant']
}
# 

mlp_gs = GridSearchCV(
    estimator=mlp_clf, 
    param_grid=param_grid, 
    cv=5,
    refit=True,
    return_train_score=True,
    verbose=5
)
mlp_gs.fit(X_train, np.ravel(Y_train))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END activation=relu, hidden_layer_sizes=(100,), solver=sgd;, score=(train=0.653, test=0.653) total time=   4.3s
[CV 2/5] END activation=relu, hidden_layer_sizes=(100,), solver=sgd;, score=(train=0.653, test=0.654) total time=   4.5s
[CV 3/5] END activation=relu, hidden_layer_sizes=(100,), solver=sgd;, score=(train=0.653, test=0.652) total time=   4.4s
[CV 4/5] END activation=relu, hidden_layer_sizes=(100,), solver=sgd;, score=(train=0.654, test=0.655) total time=   4.6s
[CV 5/5] END activation=relu, hidden_layer_sizes=(100,), solver=sgd;, score=(train=0.653, test=0.653) total time=   4.8s
[CV 1/5] END activation=relu, hidden_layer_sizes=(100,), solver=adam;, score=(train=0.807, test=0.810) total time=  10.8s
[CV 2/5] END activation=relu, hidden_layer_sizes=(100,), solver=adam;, score=(train=0.699, test=0.703) total time=  18.0s
[CV 3/5] END activation=relu, hidden_layer_sizes=(100,), solver=adam;, score=(train=0.805,

c:\Users\janba\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
c:\Users\janba\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
c:\Users\janba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 4/5] END activation=relu, hidden_layer_sizes=(50, 50, 50), solver=sgd;, score=(train=0.361, test=0.361) total time= 2.0min
[CV 5/5] END activation=relu, hidden_layer_sizes=(50, 50, 50), solver=sgd;, score=(train=0.639, test=0.639) total time=   6.5s
[CV 1/5] END activation=relu, hidden_layer_sizes=(50, 50, 50), solver=adam;, score=(train=0.719, test=0.722) total time=  24.5s
[CV 2/5] END activation=relu, hidden_layer_sizes=(50, 50, 50), solver=adam;, score=(train=0.721, test=0.721) total time=  31.0s
[CV 3/5] END activation=relu, hidden_layer_sizes=(50, 50, 50), solver=adam;, score=(train=0.717, test=0.717) total time=  22.7s
[CV 4/5] END activation=relu, hidden_layer_sizes=(50, 50, 50), solver=adam;, score=(train=0.722, test=0.721) total time=  23.3s
[CV 5/5] END activation=relu, hidden_layer_sizes=(50, 50, 50), solver=adam;, score=(train=0.720, test=0.719) total time=  23.2s
[CV 1/5] END activation=logistic, hidden_layer_sizes=(100,), solver=sgd;, score=(train=0.797, test=0.799) 

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['relu', 'logistic', 'tanh'],
                         'hidden_layer_sizes': [(100,), (50, 50, 50)],
                         'solver': ['sgd', 'adam']},
             return_train_score=True, verbose=5)

In [9]:
from sklearn.metrics import accuracy_score

y_pred_gs = mlp_gs.predict(X_test)
accuracy_score(Y_test, y_pred_gs)

0.8100972563592957

In [10]:
mlp_gs.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__activation': 'relu',
 'estimator__alpha': 0.0001,
 'estimator__batch_size': 'auto',
 'estimator__beta_1': 0.9,
 'estimator__beta_2': 0.999,
 'estimator__early_stopping': False,
 'estimator__epsilon': 1e-08,
 'estimator__hidden_layer_sizes': (100,),
 'estimator__learning_rate': 'constant',
 'estimator__learning_rate_init': 0.001,
 'estimator__max_fun': 15000,
 'estimator__max_iter': 200,
 'estimator__momentum': 0.9,
 'estimator__n_iter_no_change': 10,
 'estimator__nesterovs_momentum': True,
 'estimator__power_t': 0.5,
 'estimator__random_state': None,
 'estimator__shuffle': True,
 'estimator__solver': 'adam',
 'estimator__tol': 0.0001,
 'estimator__validation_fraction': 0.1,
 'estimator__verbose': False,
 'estimator__warm_start': False,
 'estimator': MLPClassifier(),
 'n_jobs': None,
 'param_grid': {'activation': ['relu', 'logistic', 'tanh'],
  'hidden_layer_sizes': [(100,), (50, 50, 50)],
  'solver': ['sgd', 'adam']},
 'pre_dispatch': '2*n_jo